In [23]:
//Im Terminal einmal npm install mongodb ausführen damit das Notebook klappt bei Rückfragen einfach einmal melden - Leonard Kipar
let ObjectId = require("mongodb").ObjectId;
let MongoClient = require("mongodb").MongoClient;

SyntaxError: Identifier 'ObjectId' has already been declared

In [24]:
let mongourl = "mongodb://127.0.0.1:27017";

SyntaxError: Identifier 'mongourl' has already been declared

In [3]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let movie = await movies.findOne({"title":"The Big Short"});
    
    console.log(movie);
    
    db.close();
});

{
  _id: new ObjectId("639ef6c204bd8ad9cd853cbb"),
  imdbID: '1596363',
  title: 'The Big Short',
  year: '2016',
  genre: 'Drama',
  director: 'Adam McKay',
  writer: 'Michael Lewis (based on the book by), Adam McKay (screenplay), Charles Randolph (screenplay)',
  cast: 'Brad Pitt, Christian Bale, Selena Gomez, Karen Gillan',
  plot: 'The men who made millions from a global economic meltdown.',
  fullplot: 'The men who made millions from a global economic meltdown.',
  country: 'USA',
  lastupdated: '2015-09-12 00:20:34.633000000',
  type: 'movie'
}


In [4]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let languages = await movies.aggregate([
        { 
            $match: {  
                language : { $regex: /^English/ }, 
            },
        },
        {
            $group: {
                _id: "$language",
                totalQty: { $count: {} },
            },
        },
        { 
            $sort: { totalQty: -1 } 
        },
    ]).toArray();
    console.log(languages);
    db.close();
});

[
  { _id: 'English', totalQty: 25325 },
  { _id: 'English, Spanish', totalQty: 728 },
  { _id: 'English, French', totalQty: 584 },
  { _id: 'English, German', totalQty: 288 },
  { _id: 'English, Italian', totalQty: 263 },
  { _id: 'English, Russian', totalQty: 159 },
  { _id: 'English, Japanese', totalQty: 159 },
  { _id: 'English, Latin', totalQty: 61 },
  { _id: 'English, Mandarin', totalQty: 60 },
  { _id: 'English, French, German', totalQty: 55 },
  { _id: 'English, Portuguese', totalQty: 53 },
  { _id: 'English, Arabic', totalQty: 50 },
  { _id: 'English, Cantonese', totalQty: 41 },
  { _id: 'English, German, French', totalQty: 40 },
  { _id: 'English, French, Spanish', totalQty: 39 },
  { _id: 'English, Swedish', totalQty: 38 },
  { _id: 'English, Hebrew', totalQty: 36 },
  { _id: 'English, French, Italian', totalQty: 32 },
  { _id: 'English, American Sign Language', totalQty: 30 },
  { _id: 'English, Spanish, French', totalQty: 29 },
  { _id: 'English, Vietnamese', totalQty: 28

In [6]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let languages = await movies.aggregate([
        {
            $sortByCount: "$language",
        }
    ]).toArray();
    console.log(languages);
    db.close();
});

[
  { _id: 'English', count: 25325 },
  { _id: 'French', count: 1784 },
  { _id: 'Italian', count: 1480 },
  { _id: 'Japanese', count: 1290 },
  { _id: null, count: 1115 },
  { _id: 'Spanish', count: 875 },
  { _id: 'Russian', count: 777 },
  { _id: 'English, Spanish', count: 728 },
  { _id: 'German', count: 674 },
  { _id: 'English, French', count: 584 },
  { _id: 'Hindi', count: 498 },
  { _id: 'Korean', count: 377 },
  { _id: 'Finnish', count: 349 },
  { _id: 'Swedish', count: 291 },
  { _id: 'English, German', count: 288 },
  { _id: 'Mandarin', count: 287 },
  { _id: 'English, Italian', count: 263 },
  { _id: 'Portuguese', count: 251 },
  { _id: 'French, English', count: 246 },
  { _id: 'Polish', count: 203 },
  { _id: 'Dutch', count: 199 },
  { _id: 'Cantonese', count: 170 },
  { _id: 'English, Russian', count: 159 },
  { _id: 'English, Japanese', count: 159 },
  { _id: 'Danish', count: 157 },
  { _id: 'Turkish', count: 155 },
  { _id: 'Japanese, English', count: 134 },
  { _id: '

In [8]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let cnt1Pipeline = [ { '$count': 'year' } ];
    let cnt2Pipeline = [ { '$count': 'director' } ];
    let cnt3Pipeline = [ { '$count': 'writer' } ];
    let languages = await movies.aggregate(cnt1Pipeline).toArray();
    console.log(languages);
    db.close();
});

[ { year: 46014 } ]


In [10]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let cnt1Pipeline = [{ 
        $group: {
            _id: "Jahre",
            minYear: { $min: "$year" },
            maxYear: { $max: "$year" },
        }
    }];
    let languages = await movies.aggregate(cnt1Pipeline).toArray();
    console.log(languages);
    db.close();
});

[ { _id: 'Jahre', minYear: '1874', maxYear: '2019' } ]


In [28]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let years = await movies.aggregate([
        {
            $project: {
                _id: 1,
                year: 1
            },
        }
    ]).toArray();
    console.log(years.length);
    for(entries of years) {
        if(entries.year.length > 4) {
            await movies.updateOne({_id: ObjectId(entries._id.toString())},{$set: {year: parseInt(entries.year.split("�")[0])}})
        } else {
            await movies.updateOne({_id: ObjectId(entries._id.toString())},{$set: {year: parseInt(entries.year)}})
        }
    }
    db.close();
});

46014


In [30]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let cnt1Pipeline = [
        {
            $project: {
                //_id: 1,
                year: 1
            },
        }
    ];
    let years = await movies.aggregate(cnt1Pipeline).toArray();
    for(entries of years) {
        //console.log(entries.year.toString().length)
        if(entries.year.toString().length > 4) {
            console.log(entries.year);
        }
    }
    db.close();
});

In [31]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let year2016 = [
        {
            $match: { year: 2016 }
        },
        {
            $sortByCount: "$year",
        }
    ];
    let years = await movies.aggregate(year2016).toArray();
    console.log(years);
    db.close();
});

[ { _id: 2016, count: 523 } ]


In [36]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let list = await movies.distinct("year");

    
    //Aggregate
    let start_timer = performance.now();
    for(yr of list) {
        let years = await movies.aggregate([
            {
                $match: {
                    year: yr
                }
            }
        ]);
    }
    let stop_timer = performance.now();
    console.log(`Aggregate Time: ${stop_timer-start_timer}ms`);
    //
    
    //Find
    let start_timer2 = performance.now();
    for(yr of list) {
        let find = await movies.find({year: yr});
    }
    let stop_timer2 = performance.now();
    console.log(`Find Time: ${stop_timer2-start_timer2}ms`);
    //
    
    db.close();
});

Aggregate Time: 2.1950550000183284ms
Find Time: 0.8175679999403656ms


In [38]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let movies2016 = await movies.aggregate([
        {
            $match: {
                year: 2016
            },
        },
        {  
            $group: {
                _id: "$rating",
                count: { $count: {} }
            },
        },
        {
            $sort: { count: -1 }
        }
    ]).toArray();
    console.log(movies2016);
    db.close();
});

[
  { _id: null, count: 517 },
  { _id: 'R', count: 5 },
  { _id: 'PG-13', count: 1 }
]


In [40]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let shortSince2000 = await movies.aggregate([
        {
            $match: {
                genre: { $regex: /Short/ },
                year: { $gte: 2000 }
            },
        },
        {  
            $group: {
                _id: "Short",
                count: { $count: {} }
            },
        }
    ]).toArray();
    console.log(shortSince2000);
    db.close();
});

[ { _id: 'Short', count: 559 } ]


In [42]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let shortSince2000 = await movies.aggregate([
        {
            $match: {
                genre: {$regex: /Short/ },
                year: { $gte: 2000 }
            },
        },
        {  
            $project: {
                title: "$title",
                year: "$year",
                genre: "$genre"
            },
        },
        {
            $sort: {
                title: 1
            }
        },
        {
            $out: { db: "ShortMovies", coll: "2000er"}
        }
    ]).toArray();
    //console.log(shortSince2000);
    //await db.db("ShortMovies").collection("2000er").insertMany(shortSince2000);
    db.close();
});